In [1]:
import os
import itertools
import numpy as np
import pandas as pd

In [2]:
import sys
sys.path.append('../src/')

%load_ext autoreload
%autoreload 2
# Importing our custom module(s)
import folds
import utils

In [3]:
'''Code to view category prevalence'''
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
df = pd.read_csv(os.path.join(dataset_path, 'HAM10000_metadata'))
df.dx.value_counts()/df.shape[0]

nv       0.669496
mel      0.111133
bkl      0.109735
bcc      0.051323
akiec    0.032651
vasc     0.014179
df       0.011483
Name: dx, dtype: float64

In [4]:
'''Code to create labels.csv'''
def label_to_category(label, categories):
    if label not in categories: return [0]
    else: return [categories.index(label)+1]

dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
df = pd.read_csv(os.path.join(dataset_path, 'HAM10000_metadata'))
categories = ['bkl', 'nv', 'df', 'mel', 'vasc', 'bcc', 'akiec']
categories = ['bkl', 'nv', 'mel'] # categories based off prevalence
df['label'] = df['dx'].apply(lambda label: label_to_category(label, categories))
df['path'] = df['image_id'].apply(lambda image_id: os.path.join(dataset_path, 'train/{}.jpg'.format(image_id)))
temp_df = df[['lesion_id', 'label', 'path']].set_index('lesion_id')
temp_df.to_csv(os.path.join(dataset_path, 'train/labels.csv'))
temp_df.head()

,label,path
lesion_id,,
HAM_0000118,[1],/cluster/tufts/hugheslab/eharve06/HAM10000/tra...
HAM_0000118,[1],/cluster/tufts/hugheslab/eharve06/HAM10000/tra...
HAM_0002730,[1],/cluster/tufts/hugheslab/eharve06/HAM10000/tra...
HAM_0002730,[1],/cluster/tufts/hugheslab/eharve06/HAM10000/tra...
HAM_0001466,[1],/cluster/tufts/hugheslab/eharve06/HAM10000/tra...


In [5]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
# TODO: Remove ISIC_0035068.jpg from ISIC2018_Task3_Test_GroundTruth.csv,
#       the image is missing from ISIC2018_Task3_Test_Images.zip
df = pd.read_csv(os.path.join(dataset_path, 'ISIC2018_Task3_Test_GroundTruth.csv'))
categories = ['bkl', 'nv', 'df', 'mel', 'vasc', 'bcc', 'akiec']
categories = ['bkl', 'nv', 'mel'] # categories based off prevalence
df['label'] = df['dx'].apply(lambda label: label_to_category(label, categories))
df['path'] = df['image_id'].apply(lambda image_id: os.path.join(dataset_path, 'test/{}.jpg'.format(image_id)))
temp_df = df[['lesion_id', 'label', 'path']].set_index('lesion_id')
temp_df.to_csv(os.path.join(dataset_path, 'test/labels.csv'))
temp_df.head()

,label,path
lesion_id,,
HAMTEST_0000000,[2],/cluster/tufts/hugheslab/eharve06/HAM10000/tes...
HAMTEST_0000001,[2],/cluster/tufts/hugheslab/eharve06/HAM10000/tes...
HAMTEST_0000002,[1],/cluster/tufts/hugheslab/eharve06/HAM10000/tes...
HAMTEST_0000003,[2],/cluster/tufts/hugheslab/eharve06/HAM10000/tes...
HAMTEST_0000004,[2],/cluster/tufts/hugheslab/eharve06/HAM10000/tes...


In [6]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
augmented_train_dataset, train_dataset, val_or_test_dataset = utils.get_ham10000_datasets(dataset_path, n=100, tune=True, random_state=1001)
[label for image, label in val_or_test_dataset]

[tensor(2),
 tensor(2),
 tensor(2),
 tensor(2),
 tensor(2),
 tensor(3),
 tensor(1),
 tensor(2),
 tensor(2),
 tensor(3),
 tensor(1),
 tensor(2),
 tensor(2),
 tensor(2),
 tensor(2),
 tensor(2),
 tensor(2),
 tensor(2)]

In [3]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000'
lr_0s = np.logspace(-1, -4, num=4)
ns = [100, 1000]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
random_states = [1001, 2001, 3001]
prior_type = 'nonlearned'
wandb_project = 'tuned_HAM10000'
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
count = -1
for lr_0, n, random_state, weight_decay in itertools.product(lr_0s, ns, random_states, weight_decays):
    count += 1
    model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
    .format(prior_type, lr_0, n, random_state, weight_decay)
    print('    "python ../src/HAM10000_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --prior_type=\'{}\' --random_state={} --tune --wandb --wandb_project=\'{}\' --weight_decay={}"'\
          .format(dataset_path, experiments_path, lr_0, model_name, n, prior_path, prior_type, random_state, wandb_project, weight_decay))
print(count)

    "python ../src/HAM10000_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/HAM10000' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000' --lr_0=0.1 --model_name='nonlearned_lr_0=0.1_n=100_random_state=1001_weight_decay=0.01' --n=100 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='nonlearned' --random_state=1001 --tune --wandb --wandb_project='tuned_HAM10000' --weight_decay=0.01"
    "python ../src/HAM10000_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/HAM10000' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000' --lr_0=0.1 --model_name='nonlearned_lr_0=0.1_n=100_random_state=1001_weight_decay=0.001' --n=100 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='nonlearned' --random_state=1001 --tune --wandb --wandb_project='tuned_HAM10000' --weight_decay=0.001"
    "python ../src/HAM10000_main.py --da

In [4]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000'
lr_0s = np.logspace(-1, -4, num=4)
ns = [100, 1000]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
prior_type = 'adapted'
random_states = [1001, 2001, 3001]
wandb_project = 'tuned_HAM10000'
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
count = -1
for lr_0, n, random_state, weight_decay in itertools.product(lr_0s, ns, random_states, weight_decays):
    count += 1
    model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
    .format(prior_type, lr_0, n, random_state, weight_decay)
    print('    "python ../src/HAM10000_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --prior_type=\'{}\' --random_state={} --tune --wandb --wandb_project=\'{}\' --weight_decay={}"'\
          .format(dataset_path, experiments_path, lr_0, model_name, n, prior_path, prior_type, random_state, wandb_project, weight_decay))
print(count)

    "python ../src/HAM10000_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/HAM10000' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000' --lr_0=0.1 --model_name='adapted_lr_0=0.1_n=100_random_state=1001_weight_decay=0.01' --n=100 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='adapted' --random_state=1001 --tune --wandb --wandb_project='tuned_HAM10000' --weight_decay=0.01"
    "python ../src/HAM10000_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/HAM10000' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000' --lr_0=0.1 --model_name='adapted_lr_0=0.1_n=100_random_state=1001_weight_decay=0.001' --n=100 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='adapted' --random_state=1001 --tune --wandb --wandb_project='tuned_HAM10000' --weight_decay=0.001"
    "python ../src/HAM10000_main.py --dataset_path='

In [5]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000'
lr_0s = np.logspace(-1, -4, num=4)
ns = [1000]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
prior_scales = np.logspace(0, 9, num=10)
prior_type = 'learned'
random_states = [1001, 2001, 3001]
wandb_project = 'tuned_HAM10000'
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
count = -1
for lr_0, n, prior_scale, random_state, weight_decay in itertools.product(lr_0s, ns, prior_scales, random_states, weight_decays):
    model_name = '{}_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
    .format(prior_type, lr_0, n, prior_scale, random_state, weight_decay)
    count += 1
    print('    "python ../src/HAM10000_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --prior_scale={} --prior_type=\'{}\' --random_state={} --tune --wandb --wandb_project=\'{}\' --weight_decay={}"'\
          .format(dataset_path, experiments_path, lr_0, model_name, n, prior_path, prior_scale, prior_type, random_state, wandb_project, weight_decay))
print(count)

    "python ../src/HAM10000_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/HAM10000' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000' --lr_0=0.1 --model_name='learned_lr_0=0.1_n=1000_prior_scale=1.0_random_state=1001_weight_decay=0.01' --n=1000 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_scale=1.0 --prior_type='learned' --random_state=1001 --tune --wandb --wandb_project='tuned_HAM10000' --weight_decay=0.01"
    "python ../src/HAM10000_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/HAM10000' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000' --lr_0=0.1 --model_name='learned_lr_0=0.1_n=1000_prior_scale=1.0_random_state=1001_weight_decay=0.001' --n=1000 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_scale=1.0 --prior_type='learned' --random_state=1001 --tune --wandb --wandb_project='tuned_HAM10000' --

In [14]:
def get_df(path):
    df = pd.read_csv(path, index_col='Unnamed: 0')
    return df

def get_val_nll(df):
    return df.val_or_test_nll.values[-1]

def get_val_acc(df):
    return df.val_or_test_acc.values[-1]

def get_last_epoch(df):
    return df.iloc[-1]

def get_hyperparameters(experiments_path, lr_0s, ns, prior_scales, prior_type, random_states, weight_decays):
    columns = ['lr_0', 'n', 'prior_scale', 'prior_type', 'random_state', 'val_acc', 'weight_decay']
    df = pd.DataFrame(columns=columns)
    for n, prior_scale, random_state in itertools.product(ns, prior_scales, random_states):
        best_val_nll = np.inf
        best_hyperparameters = None
        for lr_0, weight_decay in itertools.product(lr_0s, weight_decays):
            if prior_scale:
                model_name = '{}_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
                .format(prior_type, lr_0, n, prior_scale, random_state, weight_decay)
            else:
                model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
                .format(prior_type, lr_0, n, random_state, weight_decay)
            path =  '{}/{}.csv'.format(experiments_path, model_name)
            val_nll = get_val_nll(get_df(path))
            val_acc = get_val_acc(get_df(path))
            if val_nll < best_val_nll: best_val_nll = val_nll; best_hyperparameters = [lr_0, n, prior_scale, prior_type, random_state, val_acc, weight_decay]
        df.loc[df.shape[0]] = best_hyperparameters
    return df

def get_best_hyperparameters(experiments_path, lr_0s, ns, prior_scales, prior_type, random_states, weight_decays):
    columns = ['lr_0', 'n', 'prior_scale', 'prior_type', 'random_state', 'val_acc', 'weight_decay']
    df = pd.DataFrame(columns=columns)
    for n, random_state in itertools.product(ns, random_states):
        best_val_nll = np.inf
        best_hyperparameters = None
        for lr_0, prior_scale, weight_decay in itertools.product(lr_0s, prior_scales, weight_decays):
            if prior_scale:
                model_name = '{}_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
                .format(prior_type, lr_0, n, prior_scale, random_state, weight_decay)
            else:
                model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
                .format(prior_type, lr_0, n, random_state, weight_decay)
            path =  '{}/{}.csv'.format(experiments_path, model_name)
            val_nll = get_val_nll(get_df(path))
            val_acc = get_val_acc(get_df(path))
            if val_nll < best_val_nll: best_val_nll = val_nll; best_hyperparameters = [lr_0, n, prior_scale, prior_type, random_state, val_acc, weight_decay]
        df.loc[df.shape[0]] = best_hyperparameters
    return df

def get_results(df, experiments_path):
    columns = ['n', 'prior_scale', 'prior_type', 'random_state', 'test_acc', 'test_loss', 'test_nll', 
           'test_prior', 'train_acc', 'train_loss', 'train_nll', 'train_prior']
    results = pd.DataFrame(columns=columns)
    for row_index, row in df.iterrows():
        if row.prior_scale:
            model_name = '{}_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
            .format(row.prior_type, row.lr_0, row.n, row.prior_scale, row.random_state, row.weight_decay)
        else:
            model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
            .format(row.prior_type, row.lr_0, row.n, row.random_state, row.weight_decay)
        path =  '{}/{}.csv'.format(experiments_path, model_name)
        last_epoch = get_last_epoch(get_df(path))
        results_row = [int(row.n), row.prior_scale, row.prior_type, int(row.random_state), 
                       last_epoch.val_or_test_acc, last_epoch.val_or_test_loss, 
                       last_epoch.val_or_test_nll, last_epoch.val_or_test_prior, 
                       last_epoch.train_acc, last_epoch.train_loss, 
                       last_epoch.train_nll, last_epoch.train_prior]
        results.loc[results.shape[0]] = results_row
    return results

In [15]:
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000'
lr_0s = np.logspace(-1, -4, num=4)
ns = [100, 1000]
prior_scales = [None]
prior_type = 'nonlearned'
random_states = [1001, 2001, 3001]
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
nonlearned_hyperparameters = get_best_hyperparameters(experiments_path, lr_0s, ns, prior_scales, prior_type, random_states, weight_decays)
nonlearned_hyperparameters

,lr_0,n,prior_scale,prior_type,random_state,val_acc,weight_decay
0,0.0001,100,None,nonlearned,1001,0.610491,0.010000
1,0.0001,100,None,nonlearned,2001,0.383929,0.000010
2,0.0001,100,None,nonlearned,3001,0.820238,0.000001
3,0.0001,1000,None,nonlearned,1001,0.858887,0.010000
4,0.0001,1000,None,nonlearned,2001,0.885370,0.010000
5,0.0100,1000,None,nonlearned,3001,0.876476,0.010000


In [16]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_HAM10000'
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
wandb_project = 'retrained_HAM10000'
count = -1
for row_index, row in nonlearned_hyperparameters.iterrows():
    if row.prior_scale:
        model_name = '{}_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
        .format(row.prior_type, row.lr_0, row.n, row.prior_scale, row.random_state, row.weight_decay)
        if not os.path.exists('{}/{}.csv'.format(experiments_path, model_name)):
            count += 1
            print('    "python ../src/HAM10000_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --prior_scale={} --prior_type=\'{}\' --random_state={} --wandb --wandb_project=\'{}\' --weight_decay={}"'\
                  .format(dataset_path, experiments_path, row.lr_0, model_name, int(row.n), prior_path, row.prior_scale, row.prior_type, int(row.random_state), wandb_project, row.weight_decay))
    else:
        model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
        .format(row.prior_type, row.lr_0, row.n, row.random_state, row.weight_decay)
        if not os.path.exists('{}/{}.csv'.format(experiments_path, model_name)):
            count += 1
            print('    "python ../src/HAM10000_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --prior_type=\'{}\' --random_state={} --wandb --wandb_project=\'{}\' --weight_decay={}"'\
                  .format(dataset_path, experiments_path, row.lr_0, model_name, int(row.n), prior_path, row.prior_type, int(row.random_state), wandb_project, row.weight_decay))
print(count)

-1


In [17]:
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000'
lr_0s = np.logspace(-1, -4, num=4)
ns = [100, 1000]
prior_scales = [None]
prior_type = 'adapted'
random_states = [1001, 2001, 3001]
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
adapted_hyperparameters = get_best_hyperparameters(experiments_path, lr_0s, ns, prior_scales, prior_type, random_states, weight_decays)
adapted_hyperparameters

,lr_0,n,prior_scale,prior_type,random_state,val_acc,weight_decay
0,0.0001,100,None,adapted,1001,0.610491,0.00001
1,0.0001,100,None,adapted,2001,0.383929,0.01000
2,0.0001,100,None,adapted,3001,0.820238,0.01000
3,0.0001,1000,None,adapted,1001,0.859050,0.01000
4,0.0001,1000,None,adapted,2001,0.886299,0.01000
5,0.0001,1000,None,adapted,3001,0.854957,0.01000


In [18]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_HAM10000'
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
wandb_project = 'retrained_HAM10000'
count = -1
for row_index, row in adapted_hyperparameters.iterrows():
    if row.prior_scale:
        model_name = '{}_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
        .format(row.prior_type, row.lr_0, row.n, row.prior_scale, row.random_state, row.weight_decay)
        if not os.path.exists('{}/{}.csv'.format(experiments_path, model_name)):
            count += 1
            print('    "python ../src/HAM10000_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --prior_scale={} --prior_type=\'{}\' --random_state={} --wandb --wandb_project=\'{}\' --weight_decay={}"'\
                  .format(dataset_path, experiments_path, row.lr_0, model_name, int(row.n), prior_path, row.prior_scale, row.prior_type, int(row.random_state), wandb_project, row.weight_decay))
    else:
        model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
        .format(row.prior_type, row.lr_0, row.n, row.random_state, row.weight_decay)
        if not os.path.exists('{}/{}.csv'.format(experiments_path, model_name)):
            count += 1
            print('    "python ../src/HAM10000_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --prior_type=\'{}\' --random_state={} --wandb --wandb_project=\'{}\' --weight_decay={}"'\
                  .format(dataset_path, experiments_path, row.lr_0, model_name, int(row.n), prior_path, row.prior_type, int(row.random_state), wandb_project, row.weight_decay))
print(count)

-1


In [19]:
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000'
lr_0s = np.logspace(-1, -4, num=4)
ns = [100, 1000]
prior_scales = np.logspace(0, 9, num=10)
prior_type = 'learned'
random_states = [1001, 2001, 3001]
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
learned_hyperparameters = get_best_hyperparameters(experiments_path, lr_0s, ns, prior_scales, prior_type, random_states, weight_decays)
learned_hyperparameters

,lr_0,n,prior_scale,prior_type,random_state,val_acc,weight_decay
0,0.0001,100,100000000.0,learned,1001,0.610491,0.0100
1,0.0100,100,10.0,learned,2001,0.473214,0.0010
2,0.0001,100,1.0,learned,3001,0.820238,0.0001
3,0.0100,1000,1000000.0,learned,1001,0.888877,0.0100
4,0.0001,1000,100.0,learned,2001,0.885955,0.0100
5,0.0100,1000,100.0,learned,3001,0.879465,0.0100


In [20]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_HAM10000'
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
wandb_project = 'retrained_HAM10000'
count = -1
for row_index, row in learned_hyperparameters.iterrows():
    if row.prior_scale:
        model_name = '{}_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
        .format(row.prior_type, row.lr_0, row.n, row.prior_scale, row.random_state, row.weight_decay)
        if not os.path.exists('{}/{}.csv'.format(experiments_path, model_name)):
            count += 1
            print('    "python ../src/HAM10000_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --prior_scale={} --prior_type=\'{}\' --random_state={} --wandb --wandb_project=\'{}\' --weight_decay={}"'\
                  .format(dataset_path, experiments_path, row.lr_0, model_name, int(row.n), prior_path, row.prior_scale, row.prior_type, int(row.random_state), wandb_project, row.weight_decay))
    else:
        model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
        .format(row.prior_type, row.lr_0, row.n, row.random_state, row.weight_decay)
        if not os.path.exists('{}/{}.csv'.format(experiments_path, model_name)):
            count += 1
            print('    "python ../src/HAM10000_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --prior_type=\'{}\' --random_state={} --wandb --wandb_project=\'{}\' --weight_decay={}"'\
                  .format(dataset_path, experiments_path, row.lr_0, model_name, int(row.n), prior_path, row.prior_type, int(row.random_state), wandb_project, row.weight_decay))
print(count)

-1


In [21]:
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_HAM10000'
nonlearned_results = get_results(nonlearned_hyperparameters, experiments_path)
adapted_results = get_results(adapted_hyperparameters, experiments_path)
learned_results = get_results(learned_hyperparameters, experiments_path)
results = pd.concat([nonlearned_results, adapted_results, learned_results])
results

,n,prior_scale,prior_type,random_state,test_acc,test_loss,test_nll,test_prior,train_acc,train_loss,train_nll,train_prior
0,100,NaN,nonlearned,1001,0.750381,1.242558,1.242558,0.000000,1.000000,8.066902e-03,0.008067,0.000000e+00
1,100,NaN,nonlearned,2001,0.763326,1.328555,1.328555,0.000000,1.000000,6.217378e-03,0.006217,0.000000e+00
2,100,NaN,nonlearned,3001,0.827820,0.914008,0.914008,0.000000,1.000000,7.999104e-03,0.007999,0.000000e+00
3,1000,NaN,nonlearned,1001,0.855547,0.789375,0.789375,0.000000,0.984201,2.551718e-01,0.255172,0.000000e+00
4,1000,NaN,nonlearned,2001,0.863362,0.918982,0.918982,0.000000,0.999973,3.205305e-02,0.032053,0.000000e+00
5,1000,NaN,nonlearned,3001,0.849623,0.997314,0.997314,0.000000,1.000000,8.947629e-03,0.008948,0.000000e+00
0,100,NaN,adapted,1001,0.749754,1.261682,1.261675,0.000008,1.000000,7.478521e-03,0.007471,7.844278e-06
1,100,NaN,adapted,2001,0.762752,1.329374,1.322689,0.006685,1.000000,1.336726e-02,0.006682,6.684931e-03
2,100,NaN,adapted,3001,0.827636,0.918592,0.908662,0.009931,1.000000,1.868244e-02,0.008752,9.930908e-03
3,1000,NaN,adapted,1001,0.855007,0.799230,0.790844,0.008385,0.983847,2.656111e-01,0.257226,8.385315e-03


In [22]:
grouped_results = results.groupby(['n', 'prior_type']).agg(lambda x: tuple(x))
columns = ['test_acc', 'train_acc']
for column in columns:
    grouped_results['{}_mean'.format(column)] = grouped_results[column].apply(lambda item: np.mean(item))
    grouped_results['{}_std'.format(column)] = grouped_results[column].apply(lambda item: np.std(item))
    grouped_results['{}_min'.format(column)] = grouped_results[column].apply(lambda item: np.min(item))
    grouped_results['{}_max'.format(column)] = grouped_results[column].apply(lambda item: np.max(item))
grouped_results = grouped_results.reset_index()
grouped_results

,n,prior_type,prior_scale,random_state,test_acc,test_loss,test_nll,test_prior,train_acc,train_loss,train_nll,train_prior,test_acc_mean,test_acc_std,test_acc_min,test_acc_max,train_acc_mean,train_acc_std,train_acc_min,train_acc_max
0,100,adapted,"(nan, nan, nan)","(1001, 2001, 3001)","(0.7497538328170776, 0.7627518177032471, 0.827...","(1.2616823982671743, 1.329373971505484, 0.9185...","(1.2616745304540635, 1.3226890721753448, 0.908...","(7.844278115953786e-06, 0.0066849314607679, 0....","(1.0, 1.0, 1.0)","(0.0074785207398235, 0.0133672561496496, 0.018...","(0.0074706766754388, 0.0066823242232203, 0.008...","(7.844278115953784e-06, 0.0066849314607679, 0....",0.780047,0.034066,0.749754,0.827636,1.000000,0.000000,1.000000,1.000000
1,100,learned,"(100000000.0, 10.0, 1.0)","(1001, 2001, 3001)","(0.7498332262039185, 0.7666355967521667, 0.826...","(76089.13626633852, -3562.1311550582186, -3606...","(1.2459479925413153, 1.881782608581331, 0.8912...","(-76087.89062500001, 3564.012939453125, 3607.0...","(1.0, 1.0, 1.0)","(1149688.0, -53852.234375, -54502.62890625)","(0.0080310013145208, 0.0012754792114719, 0.012...","(-1149688.0, 53852.234375, 54502.640625)",0.781103,0.033061,0.749833,0.826841,1.000000,0.000000,1.000000,1.000000
2,100,nonlearned,"(nan, nan, nan)","(1001, 2001, 3001)","(0.7503811120986938, 0.7633255124092102, 0.827...","(1.2425584793879765, 1.328555064908248, 0.9140...","(1.2425584793879765, 1.328555064908248, 0.9140...","(0.0, 0.0, 0.0)","(1.0, 1.0, 1.0)","(0.0080669019371271, 0.0062173777259886, 0.007...","(0.0080669019371271, 0.0062173777259886, 0.007...","(0.0, 0.0, 0.0)",0.780509,0.033869,0.750381,0.827820,1.000000,0.000000,1.000000,1.000000
3,1000,adapted,"(nan, nan, nan)","(1001, 2001, 3001)","(0.8550073504447937, 0.8644254207611084, 0.874...","(0.799229583386788, 0.925009950591112, 0.80051...","(0.7908442827504963, 0.9141905942868904, 0.790...","(0.0083853146061301, 0.0108193717896938, 0.010...","(0.9838466644287108, 0.9999533295631408, 0.998...","(0.2656110631227493, 0.0455598333477973, 0.109...","(0.2572257571220397, 0.0347404615581035, 0.099...","(0.0083853146061301, 0.0108193717896938, 0.010...",0.864602,0.007906,0.855007,0.874372,0.993943,0.007182,0.983847,0.999953
4,1000,learned,"(1000000.0, 100.0, 100.0)","(1001, 2001, 3001)","(0.8612802028656006, 0.8633915185928345, 0.860...","(40314.16496990816, -3263.7312732345613, -3261...","(0.9382750694519792, 0.9152285444428792, 0.988...","(-40313.22656250001, 3264.646484375001, 3262.3...","(1.0, 0.9999691843986512, 1.0)","(60913.29296875, -4932.84775390625, -4929.4005...","(0.0086460241451859, 0.0330690816938877, 0.007...","(-60913.28515625001, 4932.880859375, 4929.4077...",0.861803,0.001145,0.860736,0.863392,0.999990,0.000015,0.999969,1.000000
5,1000,nonlearned,"(nan, nan, nan)","(1001, 2001, 3001)","(0.8555468320846558, 0.863362193107605, 0.8496...","(0.7893752272282115, 0.918981594648052, 0.9973...","(0.7893752272282115, 0.918981594648052, 0.9973...","(0.0, 0.0, 0.0)","(0.9842013120651244, 0.9999729990959167, 1.0)","(0.2551718409061432, 0.0320530468821525, 0.008...","(0.2551718409061432, 0.0320530468821525, 0.008...","(0.0, 0.0, 0.0)",0.856177,0.005627,0.849623,0.863362,0.994725,0.007441,0.984201,1.000000


In [23]:
grouped_results[['n', 'prior_type', 'test_acc_mean', 'test_acc_min', 'test_acc_max']]

,n,prior_type,test_acc_mean,test_acc_min,test_acc_max
0,100,adapted,0.780047,0.749754,0.827636
1,100,learned,0.781103,0.749833,0.826841
2,100,nonlearned,0.780509,0.750381,0.827820
3,1000,adapted,0.864602,0.855007,0.874372
4,1000,learned,0.861803,0.860736,0.863392
5,1000,nonlearned,0.856177,0.849623,0.863362
